In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gdown

# Google Drive direct download URLs
data_url = "https://drive.google.com/uc?id=1M0C3dn7_tewI-35YvH2vHdrN3ULHd0AM"
labels_url = "https://drive.google.com/uc?id=1s7H1eXU9sjpuC0p-bB0Buhs8ldmFH2J3"

# Download files
gdown.download(data_url, "grad_cam.npy", quiet=False)
gdown.download(labels_url, "labels_pred.npy", quiet=False)

# Load data
# data = np.load("grad_cam.npy")
# labels = np.load("labels_pred.npy")


Downloading...
From: https://drive.google.com/uc?id=1M0C3dn7_tewI-35YvH2vHdrN3ULHd0AM
To: /content/grad_cam.npy
100%|██████████| 28.8M/28.8M [00:00<00:00, 32.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1s7H1eXU9sjpuC0p-bB0Buhs8ldmFH2J3
To: /content/labels_pred.npy
100%|██████████| 15.5k/15.5k [00:00<00:00, 31.1MB/s]


'labels_pred.npy'

In [ ]:
def find_peak_coordinates(img):
    """Returns (row, col) of the maximum value (peak) in the 2D array."""
    return np.unravel_index(np.argmax(img), img.shape)

def calculate_distance(p1, p2):
    """Returns Euclidean distance between two (row, col) points."""
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)


In [ ]:
# Load Excel file
df = pd.read_excel("Distances.xlsx", index_col="index")

# Load data
images_data = np.load("grad_cam.npy")
labels_data = np.load("labels_pred.npy")

# Extract components
images = images_data[:, :, :, 0]
heatmaps = images_data[:, :, :, 1]
indices = labels_data[:, 0].astype(int)  # Ensure it's integer for comparison


In [ ]:
print(df.head())
print(df.columns)

       Dist_imgPeak_to_attnPeak
index                          
0                     12.369320
1                      3.162278
2                      5.830952
3                     87.132080
4                      2.236068
Index(['Dist_imgPeak_to_attnPeak'], dtype='object')


In [ ]:
computed_distances = []
match_results = []
missing = []

for idx in df.index:
    if idx in indices:
        index = np.where(indices == idx)[0][0]
        image = images[index]
        heatmap = heatmaps[index]

        img_peak = find_peak_coordinates(image)
        heatmap_peak = find_peak_coordinates(heatmap)
        distance = calculate_distance(img_peak, heatmap_peak)

        computed_distances.append(distance)
        expected_distance = df.at[idx, "Dist_imgPeak_to_attnPeak"]
        match_results.append(abs(distance - expected_distance) < 1e-4)
    else:
        computed_distances.append(None)
        match_results.append(False)
        missing.append(idx)

# Add results to DataFrame
df["Computed_Distance"] = computed_distances
df["Match"] = match_results


# Print summary
num_correct = sum(match_results)
num_total = len(match_results)
print(f"{num_correct}/{num_total} matched")


585/639 matched


In [ ]:
missing_data = df[df["Computed_Distance"].isna()]
print(f"\nResults for missing index {missing_data}:")


Results for missing index        Dist_imgPeak_to_attnPeak  Computed_Distance  Match
index                                                    
273                    16.15549                NaN  False:


In [ ]:
# print(df[df["Match"]=="False"])
print(df["Match"].value_counts())
df.head()

Match
True     585
False     54
Name: count, dtype: int64


,Dist_imgPeak_to_attnPeak,Computed_Distance,Match
index,,,
0,12.369320,12.369317,True
1,3.162278,3.162278,True
2,5.830952,5.830952,True
3,87.132080,87.132084,True
4,2.236068,2.236068,True


In [ ]:
mismatches = df[~df["Match"]]
print(mismatches)

       Dist_imgPeak_to_attnPeak  Computed_Distance  Match
index                                                    
214                    4.123106           6.324555  False
215                    6.324555           4.123106  False
216                    4.123106           2.828427  False
217                    2.828427           9.055385  False
218                    9.055385           4.242641  False
219                    4.242641           5.000000  False
220                    5.000000          23.537205  False
221                   23.537200           5.000000  False
222                    5.000000           4.242641  False
223                    4.242641           2.236068  False
224                    2.236068          15.811388  False
225                   15.811390           2.828427  False
226                    2.828427           5.830952  False
227                    5.830952           5.385165  False
228                    5.385165          11.180340  False
229           

In [ ]:
# Save results
df.to_excel("Distances_Comparison.xlsx", index=False)
